In [1]:
!pip install selenium


In [1]:
#Import the necessary packages
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time


In [2]:
service=Service(ChromeDriverManager().install())  #Initialize the selenium webdriver
options=webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])  #This will allow us to list errors if any
options.add_extension('AdBlock — block ads across the web 6.1.1.0.crx') #In order to make webscraping faster we add a adblock


In [19]:
#Loading the web driver and the website
driver = webdriver.Chrome(options=options,service=service)
driver.get('https://www.rxlist.com/drugs/alpha_c.htm')


In [20]:
#Loading beautiful soup
soup = BeautifulSoup(driver.page_source, 'html.parser')


In [21]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
k=soup.find('div',class_='AZ_results')
li=k.findAll('ul')
m=0
list_sizes=[]
for i in range(len(li)):
    s=set()
    li1=li[i]
    print(i)
    print()
    li_list=li1.findChildren()
    for i in li_list:
        s.add(i.text)
    list_sizes.append(len(s))

    

0

1

2

3

4

5

6



In [22]:
#Finding all subheadings for iteration
ul=k.find('h4').find_all('a')
ulA=len(ul)

In [23]:
print(ulA)


7


In [24]:
print(list_sizes[0])

168


In [25]:
medicines_dataF=[]
m=0
wait = WebDriverWait(driver, 30)
for i in range(0,ulA):
    s=list_sizes[i]
    print(s)
    for j in range(1,s+1):
        second = wait.until(EC.element_to_be_clickable((By.XPATH, f'//*[@id="AZ_container"]/div/ul[{i+1}]/li[{j}]/a'))) #This will make sure that page is loaded before clicking
        driver.execute_script("arguments[0].click()", second)
        soup=BeautifulSoup(driver.page_source,'html.parser') #Beautiful spup is loaded for  each webpage
        medicine_name=soup.find('h1').get_text().strip() #The h1 heading is extracted as Medicine name
        mai=soup.find('div',class_='monograph_cont')
        symptom=mai.find('p').text #The first paragraph is extracted as the use of the medicine
        dosage=mai.find_all('h4')[2].findNextSibling().text if len(mai.find_all('h4')) > 2 else "N/A" #The dosage is extracted from the 3rd h4 tag
        side=mai.findAll('li')[0:6] #The first 6 side effects are extracted
        side_effects=[]
        for effect in side:
            side_effects.append(effect.text)
        medicine_data = {
    'Medicine Name': medicine_name,
    'Use': symptom,
    'Side Effects': side_effects,
    'Dosage': dosage
}
        print(medicine_data)
        medicines_dataF.append(medicine_data)
        driver.back()
 



168


{'Medicine Name': 'Cinryze', 'Use': 'Cinryze (c1 esterase inhibitor [human]) Freeze Dried Powder is a man-made form of a protein that occurs naturally in the bloodstream and helps control inflammation in the body and is used in people with hereditary angioedema. The Berinert brand of this medication is used to treat attacks of angioedema. The Cinryze brand is used to prevent attacks of angioedema.', 'Side Effects': ['hives,', 'difficulty breathing,', 'swelling of your face, lips, tongue, or throat,', 'chest tightness,', 'fast heartbeats,', 'lightheadedness,'], 'Dosage': 'A dose of 1,000 Units Cinryze can be administered every 3 or 4 days for routine prophylaxis against angioedema attacks in HAE patients.'}
{'Medicine Name': 'Ruconest', 'Use': 'Ruconest (C1 esterase inhibitor [recombinant]) Intravenous Injection is a C1 esterase inhibitor [recombinant] indicated for the treatment of acute attacks in adult and adolescent patients with hereditary angioedema (HAE).', 'Side Effects': ['hiv

In [13]:
len(medicines_dataF)

121

In [77]:
dfrm=pd.read_csv('a-b-c-d-e-f-g-h-i-j-k-l.csv')
data=pd.DataFrame(medicines_dataF)
data1=pd.concat([dfrm,data]).reset_index(drop=True)
data1

,Medicine Name,Use,Side Effects,Dosage
0,A-Methapred,A-Methapred (methylprednisolone sodium succina...,"['muscle weakness', 'peptic ulcer', 'slow woun...",Dosage recommendations for A-Methapred vary fo...
1,Kivexa,Kivexa (abacavir and lamivudine) is a combinat...,"['hives,', 'difficulty breathing,', 'swelling ...",The recommended dose of Kivexa tablets in adul...
2,Ziagen,Ziagen (abacavir sulfate) is an antiviral medi...,"['trouble sleeping, ', 'loss of appetite, ', '...",The recommended oral dose of Ziagen for adults...
3,Epzicom,Epzicom (abacavir sulfate and lamivudine) is a...,"['headache, ', 'nausea, ', 'diarrhea, ', 'dizz...",The recommended oral dose of Epzicom for adult...
4,Trizivir,"Trizivir (abacavir sulfate, lamivudine, and zi...","['headache, ', 'tiredness, ', 'weakness, ', 'n...",The recommended oral dose of Trizivir is one t...
...,...,...,...,...
903,Meruvax,Meruvax II (rubella virus) Vaccine Live is a l...,[],The dose of Meruvax II for any age is 0.5 mL a...
904,Canasa,Canasa (mesalamine) is an anti-inflammatory dr...,"[rectal pain, , pain when inserting the suppos...",The usual dosage of Canasa 1000 mg suppositori...
905,Pentasa,Pentasa (mesalamine) is an anti-inflammatory a...,"[severe stomach pain,, stomach cramping,, bloo...",The recommended dosage of Pentasa for the indu...
906,Lialda,Lialda (mesalamine) is an anti-inflammatory ag...,"[nausea, , vomiting, , stomach cramps, , diarr...",The recommended dosage of Lialda for the induc...


In [1]:
data1.to_csv('a-b-c-d-e-f-g-h-i-j-k-l-m.csv',index=True)
    

NameError: name 'data1' is not defined

In [2]:
dfrm=pd.read_csv('a-b-c-d-e-f-g-h-i-j-k-l-m.csv')

NameError: name 'pd' is not defined